In [1]:
import pandas as pd
import geopandas as gpd
from osgeo import gdal, ogr,osr
import os
import glob
import numpy

In [5]:
#
#  After tiling process, we merge all tile in WDPA_All folder and copy this file into data server'''
#

In [5]:
%%time
# input mosaic tif file
img='protected_areas_shp/WDPA_WDOECM_WA_shp/WDPA_categorized.tif'
# define tiles path
path='protected_areas_shp/WDPA_WDOECM_WA_shp/Tiles_WDPA_WA_IUCN_CAT'
input_path=img

# name of tile image
shortname_of_continental='WA'
output_path=path+f'/{shortname_of_continental}_WDPA_tile_'
ds = gdal.Open(img)
band = ds.GetRasterBand(1)
xsize = band.XSize
ysize = band.YSize
# define your tile size
# it could be 256,512 or 1024. it depends on your scope
tile_size_x = 1024
tile_size_y = 1024
for i in range(0, xsize, tile_size_x):
    if xsize-i < tile_size_x:
        i=xsize-tile_size_x            
    for j in range(0, ysize, tile_size_y):
        if ysize-j < tile_size_y:
            j=ysize-tile_size_y

        com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(input_path)+ " " + str(output_path)  + str(i) + "_" + str(j) + ".tif"
        #print(com_string)
        os.system(com_string)
    
# with this block, we delete tiles that include just nodata
tiles=glob.glob(path+'/*')
for img in tiles:
    hdf_ds = gdal.Open(img, gdal.GA_ReadOnly)
    array_img=hdf_ds.ReadAsArray()
    #np_where_img = numpy.where((array_img<0),-32768,(array_img))
    np_where_img=array_img
    check=numpy.all(np_where_img==-32768)
    # if np_where_img all value equal nodata value
    # remove the img
    if check:
        os.remove(img)
        continue

array_img=None
np_where_img=None   



CPU times: user 88.7 ms, sys: 72.9 ms, total: 162 ms
Wall time: 3.95 s
